股吧评论数据

In [ ]:
##将获取到的股吧评论数据进行切分，确保每行每列表示一个含义
import os
import pandas as pd

# 设置文件夹路径
folder_path = 'E:/股吧爬虫/股吧爬虫/古巴爬虫5'  # 替换为你的文件夹路径

# 创建保存切分后数据的文件夹
output_folder = 'E:/股吧爬虫/股吧爬虫/古巴爬虫1/切分'
os.makedirs(output_folder, exist_ok=True)

# 遍历文件夹中的每个CSV文件
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # 构造完整的文件路径
        file_path = os.path.join(folder_path, filename)

        # 读取CSV文件
        df = pd.read_csv(file_path, header=None, names=['Data'],engine='python',encoding='utf-8')

        # 切分数据
        df_split = df['Data'].str.split('\t', expand=True)

        # 构造保存Excel文件的路径
        output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.xlsx")

        # 保存切分后的数据为Excel文件
        #df_split.to_excel(output_file_path, index=False, encoding='utf-8')
        df_split.to_excel(output_file_path, index=False)

        print(f"Processed {filename} and saved to {output_file_path}")


In [9]:
import zipfile
import os

zip_path = r'E:\股吧爬虫\股吧爬虫.zip'
extract_to = r'E:\股吧爬虫\股吧爬虫'  # 解压后存放的目标目录

os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_to)

print("已解压到：", extract_to)
print("目录列表：", os.listdir(extract_to))


已解压到： E:\股吧爬虫\股吧爬虫
目录列表： ['╣╔░╔┼└│µ']


In [11]:
import os
print(os.listdir(r"E:\股吧爬虫\股吧爬虫"))


['古巴爬虫1', '古巴爬虫2', '古巴爬虫3', '古巴爬虫4', '古巴爬虫5']


In [ ]:
import os
import pandas as pd

# 1. 配置文件夹路径
folder_path = r'E:\股吧爬虫\股吧爬虫\古巴爬虫1\切分'

# 2. 准备一个列表，用来记录每个文件是否为空
empty_files = []
nonempty_files = []

# 3. 遍历文件夹中的所有 .xlsx 和 .xls 文件
for filename in os.listdir(folder_path):
    # 3.1 跳过临时文件
    if filename.startswith('~$'):
        continue
    # 3.2 只处理 Excel 文件
    if not (filename.endswith('.xlsx') or filename.endswith('.xls')):
        continue

    file_path = os.path.join(folder_path, filename)
    try:
        # 4. 读取所有 sheet 到字典
        sheets = pd.read_excel(file_path, sheet_name=None)
    except Exception as e:
        print(f"无法读取 {filename}：{e}")
        continue

    # 5. 检查是否所有 sheet 都为空
    all_empty = True
    for sheet_name, df in sheets.items():
        if not df.empty:
            all_empty = False
            break

    if all_empty:
        # 6. 如果全空，记录并删除该文件
        empty_files.append(filename)
        os.remove(file_path)
        print(f"删除空文件：{filename}")
    else:
        nonempty_files.append(filename)

# 7. 将非空文件列表保存到新的 Excel（可选）
backup_df = pd.DataFrame({'NonEmptyFiles': nonempty_files})
backup_path = os.path.join(folder_path, '非空文件列表.xlsx')
backup_df.to_excel(backup_path, index=False)
print(f"已将 {len(nonempty_files)} 个非空文件名单保存至：{backup_path}")

# 8. 打印删除和保留情况
print(f"共删除 {len(empty_files)} 个空文件：", empty_files)
print(f"共保留 {len(nonempty_files)} 个文件，列表已保存。")

In [2]:
import os

folder = r'E:\股吧爬虫\股吧爬虫\古巴爬虫1\切分'
size_threshold = 10 * 1024  # 10 KB

candidates = []
for fn in os.listdir(folder):
    if fn.startswith('~$') or not fn.endswith(('.xlsx',' .xls')): 
        continue
    fp = os.path.join(folder, fn)
    if os.path.getsize(fp) < size_threshold:
        # 小文件，极可能是空表，直接删
        os.remove(fp)
    else:
        candidates.append(fn)

print(f"保留待检查的文件数：{len(candidates)}")


保留待检查的文件数：5321


In [3]:
import os
from openpyxl import load_workbook

folder = r'E:\股吧爬虫\股吧爬虫\古巴爬虫1\切分'
to_delete = []
to_keep = []

for fn in os.listdir(folder):
    if fn.startswith('~$') or not fn.endswith(('.xlsx','.xls')):
        continue
    fp = os.path.join(folder, fn)
    try:
        # 只读模式打开，不加载所有单元格到内存
        wb = load_workbook(fp, read_only=True, data_only=True)
    except Exception as e:
        print(f"无法打开 {fn}，跳过。")
        to_delete.append(fn)
        continue

    # 检查所有 sheet，只要有一个 sheet 行数>1 就保留
    nonempty = False
    for sheet in wb.worksheets:
        if sheet.max_row > 1:  
            nonempty = True
            break
    wb.close()

    if nonempty:
        to_keep.append(fn)
    else:
        to_delete.append(fn)
        os.remove(fp)

print(f"删除文件数：{len(to_delete)}；保留文件数：{len(to_keep)}")


删除文件数：0；保留文件数：5321


In [1]:
import os
import pandas as pd

folder_path = r'E:\股吧爬虫\股吧爬虫\古巴爬虫1\切分'
output_csv = r'E:\股吧爬虫\股吧爬虫\全部公司评论.csv'

# 找到所有 .xlsx 文件
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# 1. 用第一份文件确定列名并写入 CSV（写入 header）
first_df = pd.read_excel(os.path.join(folder_path, files[0]), dtype=str)
first_df.to_csv(output_csv, index=False, encoding='utf-8-sig')

# 2. 其余文件逐个读取并追加写入（不带 header）
for f in files[1:]:
    path = os.path.join(folder_path, f)
    df = pd.read_excel(path, dtype=str)
    df.to_csv(output_csv, mode='a', index=False, header=False, encoding='utf-8-sig')

print("所有文件已合并至 CSV：", output_csv)


所有文件已合并至 CSV： E:\股吧爬虫\股吧爬虫\全部公司评论.csv


In [ ]:
import pandas as pd

data_path = r'E:\股吧爬虫\股吧爬虫\全部公司评论.csv'
chunks = []

for chunk in pd.read_csv(
    data_path,
    skiprows=1,              # 跳过第一行
    header=0,                # 第二行作为列名
    dtype=str,
    parse_dates=['时间'],     # 指定解析“时间”列
    encoding='utf-8-sig',
    engine='python',         # 更宽松的 Python 引擎
    on_bad_lines='skip',     # 跳过格式不对的行
    chunksize=100_000        # 每次读取10万行
):
    print("读取块，行数：", len(chunk))
    chunks.append(chunk)

# 合并所有块
df = pd.concat(chunks, ignore_index=True)
print("总读取行数：", len(df))
print("列名：", df.columns.tolist())


# 4. 统计每个 Symbol 的评论数
counts = df.groupby('Symbol').size().reset_index(name='comment_count')
print(counts['comment_count'].describe())

# 5. 剔除评论数太少的公司
threshold = 30
removed = counts[counts['comment_count'] < threshold]
print(f"将剔除 {len(removed)} 家评论数 < {threshold} 的股票")

valid = counts[counts['comment_count'] >= threshold]['Symbol']
df = df[df['Symbol'].isin(valid)].reset_index(drop=True)
print("剔除后剩余总评论条数：", len(df))


In [1]:
import pandas as pd
from collections import Counter

data_path = r'E:\股吧爬虫\股吧爬虫\全部公司评论.csv'
CHUNKSIZE = 200_000  # 块大小，根据内存情况可调整

comment_counter = Counter()

# 分块读取，只读取“股票代码”列
for chunk in pd.read_csv(
    data_path,
    skiprows=1,
    header=0,
    usecols=['股票代码'],   # 只提取这一列
    dtype=str,
    encoding='utf-8-sig',
    engine='python',
    on_bad_lines='skip',
    chunksize=CHUNKSIZE
):
    # 直接累计计数
    comment_counter.update(chunk['股票代码'])

# 构建统计表
df_counts = pd.DataFrame({
    'Symbol': list(comment_counter.keys()),
    'total_comments': list(comment_counter.values())
})

# 描述统计
desc = df_counts['total_comments'].describe()
print("评论总数分布：")
print(desc)

# 剔除阈值
threshold = 30
removed = df_counts[df_counts['total_comments'] < threshold]
kept = df_counts[df_counts['total_comments'] >= threshold]
print(f"将剔除 {len(removed)} 家评论 < {threshold} 的公司")
print(f"保留 {len(kept)} 家评论 ≥ {threshold} 的公司")

# 保存统计结果
report_path = r'E:\inform\comment_count_report.csv'
df_counts.to_csv(report_path, index=False, encoding='utf-8-sig')
print("统计报告已保存至：", report_path)


评论总数分布：
count      5322.000000
mean      12158.739947
std       15101.904716
min         320.000000
25%        3760.000000
50%        7759.000000
75%       15119.750000
max      251568.000000
Name: total_comments, dtype: float64
将剔除 0 家评论 < 30 的公司
保留 5322 家评论 ≥ 30 的公司
统计报告已保存至： E:\inform\comment_count_report.csv


In [2]:
import pandas as pd
from snownlp import SnowNLP

# ——— 1. 配置 ———
file_path = r'E:\股吧爬虫\股吧爬虫\古巴爬虫1\切分\000001.xlsx'
NEG_THRESHOLD = 0.5

# ——— 2. 读取 Excel ———
# 假设你的评论正文在第 5 列（Python 索引 4），如果是“标题”列，改用 df['标题']
df = pd.read_excel(file_path, dtype=str)
text_col = df.columns[4]  
print(f"读取到列 `{text_col}` 共 {len(df)} 条原始记录")

# ——— 3. 安全情感打分函数 ———
def safe_sentiment(text: str) -> float:
    t = (text or '').strip()
    if not t:
        return 0.5
    try:
        return SnowNLP(t).sentiments
    except:
        return 0.5

# ——— 4. 对这一只股票的所有评论打分 ———
# 去掉空值
texts = df[text_col].dropna().astype(str).tolist()
scores = [safe_sentiment(t) for t in texts]

# ——— 5. 统计负面占比 ———
total = len(scores)
neg_count = sum(1 for s in scores if s < NEG_THRESHOLD)
neg_ratio = neg_count / total if total else 0

print(f"—— 单支股票 000001 评论情感分析 ——")
print(f"总评论数：{total}")
print(f"负面评论数：{neg_count}")
print(f"负面占比：{neg_ratio:.2%}")


读取到列 `4` 共 79921 条原始记录
—— 单支股票 000001 评论情感分析 ——
总评论数：79912
负面评论数：54161
负面占比：67.78%


In [3]:
import pandas as pd
from snownlp import SnowNLP

# ——— 1. 配置 ———
file_path = r'E:\股吧爬虫\股吧爬虫\古巴爬虫1\切分\000002.xlsx'
NEG_THRESHOLD = 0.5

# ——— 2. 读取 Excel ———
# 假设你的评论正文在第 5 列（Python 索引 4），如果是“标题”列，改用 df['标题']
df = pd.read_excel(file_path, dtype=str)
text_col = df.columns[4]  
print(f"读取到列 `{text_col}` 共 {len(df)} 条原始记录")

# ——— 3. 安全情感打分函数 ———
def safe_sentiment(text: str) -> float:
    t = (text or '').strip()
    if not t:
        return 0.5
    try:
        return SnowNLP(t).sentiments
    except:
        return 0.5

# ——— 4. 对这一只股票的所有评论打分 ———
# 去掉空值
texts = df[text_col].dropna().astype(str).tolist()
scores = [safe_sentiment(t) for t in texts]

# ——— 5. 统计负面占比 ———
total = len(scores)
neg_count = sum(1 for s in scores if s < NEG_THRESHOLD)
neg_ratio = neg_count / total if total else 0

print(f"—— 单支股票 000002 评论情感分析 ——")
print(f"总评论数：{total}")
print(f"负面评论数：{neg_count}")
print(f"负面占比：{neg_ratio:.2%}")


读取到列 `4` 共 82561 条原始记录
—— 单支股票 000002 评论情感分析 ——
总评论数：82553
负面评论数：41442
负面占比：50.20%


In [5]:
import pandas as pd
from snownlp import SnowNLP

# ——— 1. 配置 ———
file_path = r'E:\股吧爬虫\股吧爬虫\古巴爬虫1\切分\000004.xlsx'
NEG_THRESHOLD = 0.5

# ——— 2. 读取 Excel ———
# 假设你的评论正文在第 5 列（Python 索引 4），如果是“标题”列，改用 df['标题']
df = pd.read_excel(file_path, dtype=str)
text_col = df.columns[4]  
print(f"读取到列 `{text_col}` 共 {len(df)} 条原始记录")

# ——— 3. 安全情感打分函数 ———
def safe_sentiment(text: str) -> float:
    t = (text or '').strip()
    if not t:
        return 0.5
    try:
        return SnowNLP(t).sentiments
    except:
        return 0.5

# ——— 4. 对这一只股票的所有评论打分 ———
# 去掉空值
texts = df[text_col].dropna().astype(str).tolist()
scores = [safe_sentiment(t) for t in texts]

# ——— 5. 统计负面占比 ———
total = len(scores)
neg_count = sum(1 for s in scores if s < NEG_THRESHOLD)
neg_ratio = neg_count / total if total else 0

print(f"—— 单支股票 000004 评论情感分析 ——")
print(f"总评论数：{total}")
print(f"负面评论数：{neg_count}")
print(f"负面占比：{neg_ratio:.2%}")


读取到列 `4` 共 20161 条原始记录
—— 单支股票 000004 评论情感分析 ——
总评论数：20160
负面评论数：11928
负面占比：59.17%


In [3]:
import os

base_dir = r'E:\股吧爬虫\股吧爬虫\古巴爬虫1\切分'
all_files = os.listdir(base_dir)
print(f"切分目录下共 {len(all_files)} 个文件，它们是：")
print(all_files[:50])  # 只展示前 50 个，确认文件名格式


切分目录下共 5321 个文件，它们是：
['000001.xlsx', '000002.xlsx', '000004.xlsx', '000005.xlsx', '000006.xlsx', '000007.xlsx', '000008.xlsx', '000009.xlsx', '000010.xlsx', '000011.xlsx', '000012.xlsx', '000014.xlsx', '000016.xlsx', '000017.xlsx', '000019.xlsx', '000020.xlsx', '000021.xlsx', '000023.xlsx', '000025.xlsx', '000026.xlsx', '000027.xlsx', '000028.xlsx', '000029.xlsx', '000030.xlsx', '000031.xlsx', '000032.xlsx', '000034.xlsx', '000035.xlsx', '000036.xlsx', '000037.xlsx', '000039.xlsx', '000040.xlsx', '000042.xlsx', '000045.xlsx', '000046.xlsx', '000048.xlsx', '000049.xlsx', '000050.xlsx', '000055.xlsx', '000056.xlsx', '000058.xlsx', '000059.xlsx', '000060.xlsx', '000061.xlsx', '000062.xlsx', '000063.xlsx', '000065.xlsx', '000066.xlsx', '000068.xlsx', '000069.xlsx']
